In [1]:
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchtext.vocab import GloVe

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
import gc

C:\Users\Admin\anaconda3\envs\py311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [4]:
df.head()

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [5]:
embed_dim=50
global_vectors = GloVe(name='6B', dim=embed_dim)
glove_weights = torch.load(f".vector_cache/glove.6B.{embed_dim}d.txt.pt")
global_vectors.get_vecs_by_tokens(['and', 'i'])

tensor([[ 2.6818e-01,  1.4346e-01, -2.7877e-01,  1.6257e-02,  1.1384e-01,
          6.9923e-01, -5.1332e-01, -4.7368e-01, -3.3075e-01, -1.3834e-01,
          2.7020e-01,  3.0938e-01, -4.5012e-01, -4.1270e-01, -9.9320e-02,
          3.8085e-02,  2.9749e-02,  1.0076e-01, -2.5058e-01, -5.1818e-01,
          3.4558e-01,  4.4922e-01,  4.8791e-01, -8.0866e-02, -1.0121e-01,
         -1.3777e+00, -1.0866e-01, -2.3201e-01,  1.2839e-02, -4.6508e-01,
          3.8463e+00,  3.1362e-01,  1.3643e-01, -5.2244e-01,  3.3020e-01,
          3.3707e-01, -3.5601e-01,  3.2431e-01,  1.2041e-01,  3.5120e-01,
         -6.9043e-02,  3.6885e-01,  2.5168e-01, -2.4517e-01,  2.5381e-01,
          1.3670e-01, -3.1178e-01, -6.3210e-01, -2.5028e-01, -3.8097e-01],
        [ 1.1891e-01,  1.5255e-01, -8.2073e-02, -7.4144e-01,  7.5917e-01,
         -4.8328e-01, -3.1009e-01,  5.1476e-01, -9.8708e-01,  6.1757e-04,
         -1.5043e-01,  8.3770e-01, -1.0797e+00, -5.1460e-01,  1.3188e+00,
          6.2007e-01,  1.3779e-01,  4

In [6]:
# def get_embeddings(x):
#     return [global_vectors.get_vecs_by_tokens(i) for i in x.split()]
    
# embedded = batch_1[0].apply(lambda x: get_embeddings(x))
# embedded[0]

In [7]:
def get_tokens(x):
    return torch.tensor([global_vectors.stoi[i] if i in global_vectors.stoi else 0 for i in x.split()], dtype=torch.int64)
    
tokenized = (df[0].apply(lambda x: get_tokens(x))).to_list()
tokenized[0:3]

[tensor([    7, 10812,     1,  5466,     5,  1229, 12251,  1765, 29800,     3,
          4282,     5,     0, 11655,     5,  5298,  5988,  1588]),
 tensor([ 1896, 46266,    25,     0,  2578,   927,  1583,     3,   130,   454,
         11014,  7318]),
 tensor([    39,  31242,     44,   2052,   1369,     70,   3162,    149,     10,
              7,  12589,   6557,      1,    212, 104926,   1923,      1,    100,
             39,  28007,     66,      0,   4897,   1121,   3954,   2622,      3,
          11035,   9898,  12956,      5,  20746,    266,      6,  40920,   4000])]

In [8]:
# находим самое длинное предложение
max_len = 0
for i in tokenized:
    if len(i) > max_len:
        max_len = len(i)
print(max_len)

51


In [9]:
# заполняем обучающие данные, где не хватает длины до максимума -- добавляем нули
padded = [torch.cat([i, torch.tensor([0]*(max_len-len(i)))], dim=0) for i in tokenized]
padded[0]

tensor([    7, 10812,     1,  5466,     5,  1229, 12251,  1765, 29800,     3,
         4282,     5,     0, 11655,     5,  5298,  5988,  1588,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0])

In [10]:
features = padded
labels = df[1].to_list()

In [11]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

In [12]:
# проверка размерностей тензоров
embedding = nn.Embedding.from_pretrained(glove_weights[2])
e = embedding(X_train[0])
print(e.shape)

x = e.max(dim=1)[0]
print(x.shape)

linear1 = nn.Linear(51, 2)
l = linear1(x)
print(l.shape)

torch.Size([51, 50])
torch.Size([51])
torch.Size([2])


In [13]:
class GloveModel(nn.Module):
    def __init__(self, embed_dim, num_classes):
        super(GloveModel, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(glove_weights[2])
        self.linear1 = nn.Linear(embed_dim, num_classes)
        self.dropout = nn.Dropout(0.3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x = x.max(dim=1)[0]
        out = self.linear1(x)
        out = self.softmax(out)
        return out

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
glove_model = GloveModel(embed_dim=embed_dim, num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(glove_model.parameters(), lr=0.01)
glove_model.to(device)

GloveModel(
  (embedding): Embedding(400000, 50)
  (linear1): Linear(in_features=50, out_features=2, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (softmax): Softmax(dim=1)
)

In [15]:
def train_one_epoch(model, in_data, targets, batch_size=256):
    for i in tqdm(range(0, len(X_train), batch_size)):
        batch_x = torch.tensor(torch.stack(X_train[i:i + batch_size]), dtype=torch.int64).to(device)
        batch_y = torch.tensor(y_train[i:i + batch_size], dtype=torch.int64).to(device)
        optimizer.zero_grad()
        output = model(batch_x)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
        gc.collect()
    print(loss)

In [16]:
for i in range(50):
  train_one_epoch(glove_model, X_train, y_train)

100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:03<00:00,  6.59it/s]


tensor(0.6708, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:01<00:00, 13.56it/s]


tensor(0.6607, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:01<00:00, 13.60it/s]


...

tensor(0.6050, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:01<00:00, 12.13it/s]


tensor(0.6246, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:01<00:00, 11.79it/s]

tensor(0.6093, device='cuda:0', grad_fn=<NllLossBackward0>)


In [17]:
with torch.no_grad():
    output = glove_model(torch.tensor(torch.stack(X_test)).to(device))

output

tensor([[0.9665, 0.0335],
        [0.8149, 0.1851],
        [0.4630, 0.5370],
        ...,
        [0.4708, 0.5292],
        [0.5123, 0.4877],
        [0.1099, 0.8901]], device='cuda:0')

In [18]:
torch.tensor(y_test)[0:5]

tensor([0, 1, 0, 1, 1])

In [19]:
result = (output.cpu().argmax(dim=1)) == torch.tensor(y_test)
result.sum().item() / len(result)

0.6329479768786127